In [2]:
import os
# import dsutils
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from docarray.typing import NdArray
from docarray import BaseDoc, DocList
from FlagEmbedding import BGEM3FlagModel
from pymongo import MongoClient
from bson import ObjectId
from typing import Optional, Dict, List
from langchain_text_splitters import RecursiveCharacterTextSplitter

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class Document(BaseDoc):
    content: str
    docid: int
    title: str
    query: str
    summary: str
    url: str
    origin: str
    timestamp: dict
    page: int
    embedding: NdArray


def read_file(file_path):
    with open(file_path, 'r') as f:
        data = f.read()
    return data

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:200: UserWarning: Field name "summary" in "Document" shadows an attribute in parent "BaseDoc"
  warnings.warn(


In [4]:
client = MongoClient()
hostname = 'mongo.stockhelper-mongodb.store'
username = 'root'
password = 'financial'
client = MongoClient(hostname, username=username, password=password)
db = client['financial']
news = db.news.find({})

In [5]:
metadata = pd.DataFrame(news)
metadata = metadata.rename({'id': 'docid'}, axis=1)
# metadata['date'] = pd.to_datetime(metadata['date']).dt.strftime('%Y%m%d')
metadata.head()

,_id,timestamp,query,title,press,summary,content,url,origin
0,665eb4f4098aeb5dfb7f255d,2024-05-28 23:30:10,KB금융,"키움증권, 상장사 첫 밸류업 공시…KB금융 예고공시",연합뉴스TV,None,키움증권이 상장사 가운데 처음으로 밸류업 계획을 공시했습니다.키움증권은 오늘(28일...,https://n.news.naver.com/mnews/article/422/000...,http://www.yonhapnewstv.co.kr/MYH2024052802280...
1,665eb504098aeb5dfb7f255e,2024-05-28 17:48:39,KB금융,밸류업 대장주 KB금융…'밸류업 1호' 타이틀보다 중요한 것은,머니투데이,None,기업가치 제고계획 공시 가이드라인 확정 이후 밸류업 관련 1호 공시는 KB금융의 차...,https://n.news.naver.com/mnews/article/008/000...,http://news.mt.co.kr/mtview.php?no=20240528154...
2,665eb515098aeb5dfb7f255f,2024-05-28 17:04:19,KB금융,"KB금융, 역삼동에 ‘복합 자산관리센터’ 열어",이코노미스트,은행·증권·생명보험 업무 한 번에양종희 회장 “종합 금융의 메카 될 것”,[이코노미스트 김윤주 기자] KB금융그룹은 지난 27일 서울 강남구 역삼동 KB라이...,https://n.news.naver.com/mnews/article/243/000...,https://economist.co.kr/article/view/ecn202405...
3,665eb533098aeb5dfb7f2560,2024-05-28 15:46:07,KB금융,"양종희 KB금융 회장 “역삼 PB센터·WM 라운지, 종합 금융 메카될 것”",마이데일리,None,[마이데일리 = 구현주 기자] “KB 골드&와이즈 역삼 PB센터와 KB 스타 WM ...,https://n.news.naver.com/mnews/article/117/000...,https://mydaily.co.kr/page/view/20240528154345...
4,665eb539098aeb5dfb7f2561,2024-05-28 15:01:15,KB금융,"KB금융, 신개념 복합 자문 모델 선보인다…양종희 ""종합 금융의 메카 될 것""",더팩트,개인 및 법인고객 등 다양한 고객층에 차별화된 고객 경험 제공,KB금융그룹이 신개념 복합 자문 모델을 선보인다.KB금융그룹은 27일 KB금융그룹 ...,https://n.news.naver.com/mnews/article/629/000...,http://news.tf.co.kr/read/economy/2102111.htm


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, 
    chunk_overlap=128,
)

In [6]:
embedding_function = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False, device='cuda')

Fetching 30 files: 100%|██████████| 30/30 [00:45<00:00,  1.51s/it]
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
for doc in metadata:
    

In [10]:
pagebreak = chr(5716)
file_paths = glob('/nas/ysyoon/satchat/documents/txt/*.txt')
for file_path in tqdm(file_paths):
    docid = int(file_path.split('/')[-1].rstrip('.txt'))
    meta = metadata.loc[metadata['docid'] == docid].iloc[0].to_dict()
    
    save_path = f'{chunk_dir}/{docid}.docs'
    if os.path.exists(save_path):
        continue

    try:
        data = read_file(file_path)
    except:
        continue
        
    data = data.replace('\n\pagebreak\n', pagebreak)
    if len(data) == 0:
        continue
    
    texts = text_splitter.split_text(data)
    pages = np.cumsum([pagebreak in text for text in texts]) + 1
    texts = [text.replace(pagebreak, '\n') for text in texts]

    docs = []
    for text, page in zip(texts, pages):
        if len(text) < 128:
            continue
        docs.append(Document(text=text, page=page, **meta))
    docs = DocList[Document](docs)

    if len(docs) == 0:
        continue

    embeddings = embedding_function.encode(docs.text, return_dense=True, return_sparse=False)
    docs.embedding = embeddings['dense_vecs']
    
    docs.push(f'file://{save_path}')

100%|██████████| 64930/64930 [00:38<00:00, 1704.54it/s]


In [11]:
1

1